In [ ]:
#!pip install selenium

In [ ]:
#!pip install webdriver_manager

In [ ]:
#!pip install bs4

In [ ]:
#!pip install python-dateutil pytz

In [ ]:
#!pip install openai

In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# 크롬드라이버 셋팅
def set_chrome_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('headless')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

In [ ]:
url = 'https://www.investing.com/news/cryptocurrency-news/eth-etf-next-seasoned-vc-vance-spencer-shares-his-views-3290621'
# driver 설정
driver = set_chrome_driver(False)

# URL 요청
driver.get(url)

# aritivlePage는 신문기사의 본문입니다
article_page = driver.find_element(By.CLASS_NAME, 'articlePage')
print(article_page.text)

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv


# import os
# from openai import OpenAI

# client = OpenAI(
#     # This is the default and can be omitted
#     api_key=os.environ.get("OPENAI_API_KEY"),
# )

# def chat_func(prompt): 
#     chat_completion = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "user",
#                 "content": prompt,
#             }
#         ],
#         model="gpt-3.5-turbo",
#     )
#     return chat_completion.choices[0].message["content"]



# Investing.com의 Cryptocurrency 뉴스 섹션 URL
url = 'https://www.investing.com/news/cryptocurrency-news'

# requests.get()에 사용할 헤더, 일부 사이트는 기본 Python user-agent를 차단할 수 있음
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# URL에서 HTML 콘텐츠를 가져옴
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# #leftColumn 내의 뉴스 링크를 찾음
news_links = soup.select('#leftColumn div.textDiv a.title')

# 뉴스 본문과 날짜를 저장할 딕셔너리
news_time = []
news_article = []

for link in news_links:
    # 상대 경로를 절대 경로로 변환
    news_url = f"https://www.investing.com{link['href']}"
    news_response = requests.get(news_url, headers=headers)
    news_soup = BeautifulSoup(news_response.text, 'html.parser')
    
    # 본문 내용을 찾는 셀렉터 사용
    article_content = news_soup.select_one('#leftColumn > div.WYSIWYG.articlePage')
    
    # 날짜 정보를 찾는 셀렉터 사용
    date_info = news_soup.select_one("#leftColumn > div:nth-child(6) > span:nth-child(1)")
    
    
    temp_time = (date_info.text.strip()).replace("Published ", "")
    temp_time = temp_time.replace(" ET", "")

    temp_article = (article_content.text.strip()).replace(",", " ")
    temp_article = temp_article.replace("\n", "")
    
    # # 프롬프트 (요약해줘 + 긍/부정 감정도 분석해줘)
    # prompt = f'''
    # Summarize the paragraph below and interpret whether it is a positive or negative sentiment.

    # {article_content.text.strip()}
    # '''

    # print(chat_func(prompt))
    news_time.append(temp_time)
    news_article.append(temp_article)
    
    # 데모 목적으로 첫 번째 링크만 처리, 실제 사용 시 이 부분을 제거하세요

df = pd.DataFrame({
    'Date': news_time,
    'Content': news_article
})
df['Date'] = pd.to_datetime(df['Date'])
# df['Date'] = df['Date'].dt.tz_localize('UTC').tz_convert('Asia/Seoul')
df
csv_path = 'D:\csv\hist_news\hist_news_data.csv'  # 저장할 CSV 파일의 경로와 파일명 지정
df.to_csv(csv_path, index=False, quotechar='"', quoting=csv.QUOTE_ALL, escapechar='\\')

AttributeError: 'NoneType' object has no attribute 'text'

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

# 뉴스 본문과 날짜를 저장할 리스트 초기화
news_header = []
news_time = []
news_article = []

headers = {'User-Agent': 'Mozilla/5.0'}

# 62페이지부터 3819페이지까지 반복
for page in range(62, 64):
    # 페이지 번호를 포함한 URL
    url = f'https://www.investing.com/news/cryptocurrency-news/{page}'
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 현재 페이지 내의 뉴스 링크를 찾음
    news_links = soup.select('#leftColumn div.textDiv a.title')
    
    for link in news_links:
        # 상대 경로를 절대 경로로 변환
        news_url = f"https://www.investing.com{link['href']}"
        news_response = requests.get(news_url, headers=headers)
        news_soup = BeautifulSoup(news_response.text, 'html.parser')
        
        title_element = news_soup.select_one("#leftColumn > h1")
        if title_element:
            news_header.append(title_element.text.strip())
        
            # 본문 내용을 찾는 셀렉터 사용
            article_content = news_soup.select_one('#leftColumn > div.WYSIWYG.articlePage')

            # article_content가 None이 아닐 때만 처리를 계속 진행
            if article_content:
                exclude_element = article_content.select_one('#imgCarousel > span')
                if exclude_element:
                    exclude_element.decompose()
                
                exclude_element_relatedInstruments = article_content.select_one('div.relatedInstrumentsWrapper > div')
                if exclude_element_relatedInstruments:
                    exclude_element_relatedInstruments.decompose()
            
            # 날짜 정보를 찾는 셀렉터 사용
            date_info = news_soup.select_one("#leftColumn > div:nth-child(6) > span:nth-child(1)")
            
            if date_info and article_content:
                temp_time = (date_info.text.strip()).replace("Published ", "").replace(" ET", "")
                temp_article = (article_content.text.strip()).replace(",", " ").replace("\n", "")
                
                news_time.append(temp_time)
                news_article.append(temp_article)
        print(page,'번째 페이지 저장 완료')
        

# 데이터 프레임 생성
df = pd.DataFrame({
    'Date': news_time,
    'Header' : news_header,
    'Content': news_article
})

# 날짜 형식 변환
df['Date'] = pd.to_datetime(df['Date'])

df = df.sort_values(by='Date', ascending=True)  # 날짜 기준으로 오름차순 정렬
df = df.drop_duplicates(subset='Header', keep='first')  # 제목 기준으로 중복 제거, 가장 오래된 것을 유지

# CSV 파일로 저장
csv_path = 'D:\csv\hist_news\hist_news_data.csv'
df.to_csv(csv_path, index=False, quoting=csv.QUOTE_ALL)

62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
62 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
63 번째 페이지 저장 완료
